## People Centered COS
## CDOT Data - El Paso County 
## Investigate Bike and Pedestrian Crashes 2020 - 2024

1. Top locations 2020-2024
2. Trends over time for top N locations

### Set Up

In [10]:
pip install pandas -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install openpyxl -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

### Load Data

In [13]:
import sys  
sys.path.insert(1, '../')

from convert_old_cdot_format_to_new_format import convert_old_cdot_format_to_new_format
from cdot_clean_geography_columns import handle_missing_values_in_geography_columns, \
fill_missing_city_values, create_truncated_lat_long, combine_loc1_loc2_alphabetically 

In [14]:
# Load CDOT Data for a list of years

years = [2020, 2021, 2022, 2023, 2024]

cdot_base_path = "/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/"

loaded_data_list = []
for y in years:
    #filename = f"CDOTRM_CD_Crash_Listing_-_{y}.xlsx"
    filename = f"CDOT_ElPaso_Crash_{y}.csv"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_csv(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)

    # some folders are already converted 
    #if y < 2021:
    #    # call conversion code 
    #    year_cdot_pdf = convert_old_cdot_format_to_new_format(year_cdot_pdf)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list)

/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2020.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(8256, 143)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2021.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(10999, 111)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2022.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(10222, 111)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2023.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(10343, 111)
/Users/haley.northrup/Documents/PeopleCenteredCOS/cdot_data/fill_lat_long_elpaso_KJS/CDOT_ElPaso_Crash_2024.csv
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(9188, 111)


## Prepare Data

### Limit to Bike and Pedestrian Crashes in El Paso County 

In [15]:
print('Loaded Data Size', cdot_pdf_load.shape)

cdot_pdf_elpaso = cdot_pdf_load.loc[cdot_pdf_load['County'] == 'EL PASO']
print('El Paso County Data Size', cdot_pdf_elpaso.shape)

# bpep - bike, ped el paso
# Harmful events 
# Crash Type 
# Non motorist 
he_list = ['Pedestrian', 'Bicycle/Motorized Bicycle']
tunm_list = ['Pedestrian', 'Bicyclist', 'Other Pedestrian', 'Wheelchair', 'Scooter']
bike_ped_condition = (cdot_pdf_elpaso['First HE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['Second HE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['Third HE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['Fourth HE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['MHE'].isin(he_list)) | \
                     (cdot_pdf_elpaso['Crash Type'].isin(he_list)) | \
                     (cdot_pdf_elpaso['TU-1 NM Type'].isin(tunm_list)) | \
                     (cdot_pdf_elpaso['TU-2 NM Type'].isin(tunm_list))

cdot_pdf_bpep = cdot_pdf_elpaso.loc[bike_ped_condition]
print('CDOT El Paso bike and ped crashes size', cdot_pdf_bpep.shape)


Loaded Data Size (49008, 143)
El Paso County Data Size (49006, 143)
CDOT El Paso bike and ped crashes size (1230, 143)


### Clean Data

In [16]:
# Drop duplicates
cdot_pdf_withNans_dedup = cdot_pdf_bpep.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)
#display(cdot_pdf_withNans_dedup.head(5))

Data size after drop duplicates (1230, 143)


In [17]:
# Select columns for analysis 
analysis_columns = [
    'CUID',
    'Crash Date', 
    'City', 
    'County',
    'Crash Year',
    'Latitude', 
    'Longitude',
    'Location 1', 
    'Location 2',
    'First HE', 
    'Second HE', 
    'Third HE', 
    'Fourth HE', 
    'MHE', 
    'Crash Type', 
    'TU-1 NM Type', 
    'TU-2 NM Type',
]
cdot_pdf = cdot_pdf_withNans_dedup[analysis_columns]

# check NaN count by column 
print('nan count before handling') 
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])
print()

# handle nan, null, and other corner cases
cdot_pdf = handle_missing_values_in_geography_columns(cdot_pdf)

# check NaN count by column 
print('nan count after handling')
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])
    print(col, cdot_pdf.loc[cdot_pdf[col].isnull()].shape[0])

nan count before handling
CUID 207
CUID 207
Crash Date 0
Crash Date 0
City 105
City 105
County 0
County 0
Crash Year 0
Crash Year 0
Latitude 10
Latitude 10
Longitude 10
Longitude 10
Location 1 0
Location 1 0
Location 2 4
Location 2 4
First HE 3
First HE 3
Second HE 1087
Second HE 1087
Third HE 1192
Third HE 1192
Fourth HE 1219
Fourth HE 1219
MHE 3
MHE 3
Crash Type 26
Crash Type 26
TU-1 NM Type 707
TU-1 NM Type 707
TU-2 NM Type 554
TU-2 NM Type 554

nan count after handling
CUID 207
CUID 207
Crash Date 0
Crash Date 0
City 0
City 0
County 0
County 0
Crash Year 0
Crash Year 0
Latitude 0
Latitude 0
Longitude 0
Longitude 0
Location 1 0
Location 1 0
Location 2 4
Location 2 4
First HE 3
First HE 3
Second HE 1087
Second HE 1087
Third HE 1192
Third HE 1192
Fourth HE 1219
Fourth HE 1219
MHE 3
MHE 3
Crash Type 26
Crash Type 26
TU-1 NM Type 707
TU-1 NM Type 707
TU-2 NM Type 554
TU-2 NM Type 554


In [18]:
# Create truncated latitude and longitude for analysis 
cdot_pdf = create_truncated_lat_long(cdot_pdf, 3) 
print(cdot_pdf.columns)

Index(['CUID', 'Crash Date', 'City', 'County', 'Crash Year', 'Latitude',
       'Longitude', 'Location 1', 'Location 2', 'First HE', 'Second HE',
       'Third HE', 'Fourth HE', 'MHE', 'Crash Type', 'TU-1 NM Type',
       'TU-2 NM Type', 'Latitude_3dec', 'Longitude_3dec'],
      dtype='object')


In [19]:
# combine location 1 and location 2
# more useful to understand the intersection vs. location 1 at location 2 being different than location 2 at location 2. 
# *************************
cdot_pdf = combine_loc1_loc2_alphabetically(cdot_pdf) 

# check result
display(cdot_pdf[['Location 1', 'Location 2', 'loc1_first', 'alph_loc1_loc2']].head(5))

,Location 1,Location 2,loc1_first,alph_loc1_loc2
16,COLORADO AVE W,32ND ST S,False,32ND ST S COLORADO AVE W
77,SOUTHGATE RD,NEVADA AVE S,False,NEVADA AVE S SOUTHGATE RD
100,POWERS BLVD N,CONSTITUTION AVE,False,CONSTITUTION AVE POWERS BLVD N
132,GARDEN OF THE GODS RD W,CHESTNUT ST N,False,CHESTNUT ST N GARDEN OF THE GODS RD W
174,CHELTON RD S,ASTROZON BLVD,False,ASTROZON BLVD CHELTON RD S


## Analyze Data - Count Crashes Geographically 

- First priority is Lat Long
- Second priority is street names

### Top Crash Intersections for All Analysis Years

In [20]:
# Get the counts of crashes by latitude and longitude truncated values 
n = 9
print(f"Top {n} number of crashes in {years}")
print('Count unique truncated lat long values in dataset', 
      cdot_pdf.groupby(['Latitude_3dec', 'Longitude_3dec']).count()['City'].sort_values(ascending=False).shape[0])

# Get the lat long combos with the highest number of crashes 
top_n_latlong = pd.DataFrame(cdot_pdf.groupby(['Latitude_3dec', 'Longitude_3dec']).count()['City'].sort_values(ascending=False).head(n)) \
                    .reset_index().rename(columns={'City': 'crash_count'})
display(top_n_latlong)

# Get location information and count other crashes with same location strings 
for index, row in pd.DataFrame(top_n_latlong).iterrows():
    print()
    if row['Latitude_3dec'] != 999.999:
        # get additional count of crashes for crashes without lat-long but with the same location 1 and 2 
        associated_loc1_loc2_values = cdot_pdf.loc[(cdot_pdf['Latitude_3dec'] == row['Latitude_3dec']) & 
                     (cdot_pdf['Longitude_3dec'] == row['Longitude_3dec']), 'alph_loc1_loc2'].unique()
        extra_crashes_from_missing_ll_and_same_str = cdot_pdf.loc[(cdot_pdf['alph_loc1_loc2'].isin(associated_loc1_loc2_values)) &
                     (cdot_pdf['Latitude_3dec'] == 999.999)].shape[0]

        print(f"Top {index} number of crashes in {years}")
        print(f"Location:")
        print(f"   Associated lat {row['Latitude_3dec']}")
        print(f"   Associated long {row['Longitude_3dec']}")
        print('   first few location strings', associated_loc1_loc2_values[0:3])
        print('Crash Count:')
        print('   crash count from lat long', row['crash_count'])

Top 9 number of crashes in [2020, 2021, 2022, 2023, 2024]
Count unique truncated lat long values in dataset 853


,Latitude_3dec,Longitude_3dec,crash_count
0,999.999,999.999,19
1,38.818,-104.822,12
2,38.840,-104.776,9
3,38.827,-104.822,8
4,38.851,-104.757,8
5,38.882,-104.757,7
6,38.818,-104.824,7
7,38.814,-104.822,7
8,38.815,-104.822,7




Top 1 number of crashes in [2020, 2021, 2022, 2023, 2024]
Location:
   Associated lat 38.818
   Associated long -104.822
   first few location strings ['LAS VEGAS ST E NEVADA AVE S (NOT CDOT)' 'LAS VEGAS ST E NEVADA AVE S'
 'E LAS VEGAS ST S NEVADA AVE']
Crash Count:
   crash count from lat long 12.0

Top 2 number of crashes in [2020, 2021, 2022, 2023, 2024]
Location:
   Associated lat 38.84
   Associated long -104.776
   first few location strings ['CIRCLE DR N PLATTE AVE E' 'E PLATTE AVE N CIRCLE DR'
 ' N CIRCLE DR E PLATTE AVE']
Crash Count:
   crash count from lat long 9.0

Top 3 number of crashes in [2020, 2021, 2022, 2023, 2024]
Location:
   Associated lat 38.827
   Associated long -104.822
   first few location strings ['CIMARRON ST E NEVADA AVE S' 'E CIMARRON ST S NEVADA AVE']
Crash Count:
   crash count from lat long 8.0

Top 4 number of crashes in [2020, 2021, 2022, 2023, 2024]
Location:
   Associated lat 38.851
   Associated long -104.757
   first few location strings ['AC

### Counts by Lat Long by Year

In [27]:
# Get the lat long combos with the highest number of crashes 
gb_ct = cdot_pdf.groupby(['Crash Year', 'Latitude_3dec', 'Longitude_3dec']).count()['City'] \
                .reset_index().rename(columns={'City': 'crash_count'})

# pivot table such that the years are columns 
crash_ct_latlong_by_year = gb_ct.pivot_table(index=['Latitude_3dec', 'Longitude_3dec'], 
                                             columns='Crash Year', 
                                             values='crash_count', 
                                             fill_value=0)
crash_ct_latlong_by_year.columns = [f'Year {col} Count' for col in crash_ct_latlong_by_year.columns]
# Bring Lat and Long back as columns
crash_ct_latlong_by_year = crash_ct_latlong_by_year.reset_index() 
# Add a total column for filtering 
sum_columns = ['Year ' + str(y) + ' Count' for y in years]
crash_ct_latlong_by_year.loc[:, 'total'] = crash_ct_latlong_by_year[sum_columns].sum(axis=1)

display(crash_ct_latlong_by_year.sort_values('total', ascending=False).head(9))

,Latitude_3dec,Longitude_3dec,Year 2020 Count,Year 2021 Count,Year 2022 Count,Year 2023 Count,Year 2024 Count,total
852,999.999,999.999,2.0,8.0,7.0,2.0,0.0,19.0
232,38.818,-104.822,6.0,2.0,4.0,0.0,0.0,12.0
392,38.840,-104.776,2.0,1.0,2.0,2.0,2.0,9.0
273,38.827,-104.822,3.0,0.0,2.0,2.0,1.0,8.0
494,38.851,-104.757,2.0,1.0,2.0,1.0,2.0,8.0
609,38.882,-104.757,1.0,0.0,1.0,4.0,1.0,7.0
230,38.818,-104.824,1.0,2.0,3.0,1.0,0.0,7.0
207,38.814,-104.822,2.0,3.0,2.0,0.0,0.0,7.0
215,38.815,-104.822,1.0,4.0,1.0,1.0,0.0,7.0


In [28]:
crash_ct_latlong_by_year.sort_values('total', ascending=False) \
    .loc[crash_ct_latlong_by_year['Latitude_3dec'] != 999.999] \
    .to_csv("/Users/haley.northrup/Documents/PeopleCenteredCOS/analysis_artifacts/cdot_bikeped_2020_2024_by_counts_by_lat_long_3dec.csv", index=False)